In [ ]:
import torch
import numpy as np
import os
from typing import Tuple, Any

import cv2
import pandas as pd
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
class CustomImageDataset(Dataset):
  def __init__(self, path_to_annotation_file: str, transform: Any=None, target_transform: Any=None) -> None:
    self.path_to_annotation_file = path_to_annotation_file
    self.dataset_info = pd.read_csv(path_to_annotation_file, header=None)
    self.dataset_info.drop(self.dataset_info.columns[[0]], axis= 1, inplace= True )
    self.dataset_info.drop (index=0, axis= 0 , inplace= True )
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self) -> int:
    return len(self.dataset_info)

  def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
    path_to_image = self.dataset_info.iloc[index, 0]
    image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
    label = self.dataset_info.iloc[index, 1]

    if self.transform:
      image = self.transform(image)
    if self.target_transform:
      label = self.target_Transform(label)
      
    return image, label

In [ ]:
path_to_dataset = r"/content/drive/MyDrive/lab5/dataset/dataset"
#data_frame = pd.read_csv('annotation_2.csv')#1
#data_frame.rename(columns={'Имя классa':'class_name', 'Относительный путь': 'path_to_image'}, inplace=True)
#df1 = data_frame.reindex(columns=['class_name', 'path_to_image'])
#df1['label'] = 0
#df1.loc[df1['class_name'] == 'brown bear', 'label'] = 1
#df1.loc[df1['class_name'] == 'polar bear', 'label'] = 0
#df = df1[['path_to_image', 'label']]
#df.to_csv(os.path.join(path_to_dataset,'annotation_2.csv'))
#df['path_to_image'] = df['path_to_image'].replace(['new_data_1'], ['/content/drive/MyDrive/lab5/dataset'], regex=True)

#df

In [ ]:
custom_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Resize((224, 224)), 
                                                    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
os.path.join(path_to_dataset, "annotation_1.csv")

In [ ]:
custom_dataset = CustomImageDataset(os.path.join(path_to_dataset, "annotation_1.csv"), custom_transforms)
custom_dataset.dataset_info

In [ ]:
len(custom_dataset), custom_dataset[0][0].shape, custom_dataset[0][0].max(), custom_dataset[0][0].min()

In [ ]:
custom_dataset[0][1], custom_dataset[1500][1]

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("polar" if custom_dataset[0][1] == '0' else "brown")
plt.imshow(custom_dataset[0][0].permute(1, 2, 0).numpy()[:, :, ::-1])
plt.subplot(1, 2, 2)
plt.title("polar" if custom_dataset[1500][1] == '0' else "brown")
plt.imshow(custom_dataset[1500][0].permute(1, 2, 0).numpy()[:, :, ::-1])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
dataloader = DataLoader(custom_dataset, batch_size=4, shuffle=True)

In [ ]:
plt.figure(figsize=(10, 5))

for i_batch, sample_batched in enumerate(dataloader):
    if i_batch == 1:
      break
      
    print(i_batch, sample_batched[0].shape)

    for j in range(4):
      plt.subplot(1, 4, j + 1)
      plt.title("polar" if sample_batched[1][j] == '0' else "brown")
      plt.imshow(sample_batched[0][j].permute(1, 2, 0).numpy()[:, :, ::-1])

In [ ]:
import os
import random
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super(CNN, self).__init__()
        
        self.conv_1 = nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2)
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2)
        self.conv_3 = nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.max_pool = nn.MaxPool2d(2)
        
        self.fc_1 = nn.Linear(576, 10) # 43264 - пока что определяем экспериментальным путем (:
        self.fc_2 = nn.Linear(10, 1)
        
    def forward(self, x: torch.tensor) -> torch.tensor:
        output = self.relu(self.conv_1(x))
        output = self.max_pool(output)
        output = self.relu(self.conv_2(output))
        output = self.max_pool(output)
        output = self.relu(self.conv_3(output))
        output = self.max_pool(output)

        # print(torch.nn.Flatten()(output).shape) - определить можно, распечатав вот это

        output = torch.nn.Flatten()(output)
        output = self.relu(self.fc_1(output))
        output = torch.nn.Sigmoid()(self.fc_2(output))
        return output

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

In [ ]:
path_to_dataset = r"/content/drive/MyDrive/lab5/dataset/dataset"
path_to_train_annotation = os.path.join(path_to_dataset, "dataset_train.csv")
path_to_valid_annotation = os.path.join(path_to_dataset, "dataset_valid.csv")
path_to_test_annotation = os.path.join(path_to_dataset, "dataset_test.csv")

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model_1 = CNN().to(device)

In [ ]:
torch.cuda.is_available()

In [ ]:
custom_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Resize((224, 224)), 
                                                    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
train_dataset = CustomImageDataset(path_to_train_annotation, custom_transforms)
test_dataset = CustomImageDataset(path_to_test_annotation, custom_transforms)
valid_dataset = CustomImageDataset(path_to_valid_annotation, custom_transforms)

print(len(train_dataset), len(test_dataset), len(valid_dataset))

In [ ]:
import random
plt.figure(figsize=(10, 5))

for j in range(6):
  index = random.randint(0, 1601)
  plt.subplot(1, 6, j + 1)
  plt.title("polar" if train_dataset[index][1] == '0' else "brown")
  plt.imshow(train_dataset[index][0].permute(1, 2, 0).numpy()[:, :, ::-1])

In [ ]:
import random
plt.figure(figsize=(10, 5))

for j in range(6):
  index = random.randint(0, 201)
  plt.subplot(1, 6, j + 1)
  plt.title("polar" if test_dataset[index][1] == '0' else "brown")
  plt.imshow(test_dataset[index][0].permute(1, 2, 0).numpy()[:, :, ::-1])

In [ ]:
import random
plt.figure(figsize=(10, 5))

for j in range(6):
  index = random.randint(0, 200)
  plt.subplot(1, 6, j + 1)
  plt.title("polar" if valid_dataset[index][1] == '0' else "brown")
  plt.imshow(valid_dataset[index][0].permute(1, 2, 0).numpy()[:, :, ::-1])

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=False)

In [ ]:
optimizer = optim.Adam(params=model_1.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
epochs = 20

model_1.train()

accuracy_values = []
loss_values = []

for epoch in range(epochs):

    epoch_loss = 0
    epoch_accuracy = 0
    i = 0
    for data, label in train_dataloader:
        i+=1
        print(i)
        data = data.to(device)
        label = torch.tensor(list(int(x) for x in label))
        label = label.to(device)
        output = model_1(data)
        loss = criterion(output, label.unsqueeze(dim=1).to(torch.float))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = np.array(([1 if (1 if output[j][0].detach() >= 0.5 else 0) == int(label[j]) else 0 for j in range(4)])).mean()
        epoch_accuracy += acc / len(train_dataloader)
        epoch_loss += loss / len(train_dataloader)

    accuracy_values.append(epoch_accuracy)
    loss_values.append(epoch_loss)
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch + 1, epoch_accuracy, epoch_loss))

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(range(20), accuracy_values, color="green")
plt.plot(range(20), [float(value.detach()) for value in loss_values], color="blue")
plt.legend(["Accuracy", "Loss"])

In [ ]:
model_1.eval()

test_loss = 0
test_accuracy = 0
    
for data, label in test_dataloader:
    data = data.to(device)
    label = torch.tensor(list(int(x) for x in label))
    label = label.to(device)

    output = model_1(data)
    
    acc = np.array(([1 if (1 if output[j][0].detach() >= 0.5 else 0) == int(label[j]) else 0 for j in range(4)])).mean()
    test_accuracy += acc / len(test_dataloader)
    test_loss += float(loss.detach()) / len(test_dataloader)

In [ ]:
test_accuracy, test_loss

In [ ]:
torch.save(model_1.state_dict(), ("model_1.pt"))

In [ ]:
import random
plt.figure(figsize=(10, 5))

for j in range(2):
  index = random.randint(0, 200)
  plt.subplot(1, 2, j + 1)
  plt.title("polar" if (1 if  model_1((test_dataset[index][0].to(device)).unsqueeze(dim=0))[0] >= 0.5 else 0) == 0 else "brown")
  plt.imshow(test_dataset[index][0].permute(1, 2, 0).numpy()[:, :, ::-1])